# Introduction

For almost 200 years, we have been aware of that epidemics of diseases can be
tracked and monitored indirectly. In London, around 1854, John Snow pioneered
research into determining that [cholera outbreaks](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7150208/) could be traced to a particular
source. Discoveries such as this led to not only a focus on [treating and
cleaning](https://doi.org/10.2166/wst.2012.144) our water, but also to water being recognised as a valuable data source for disease
monitoring. For example, there has been a lot of interest aimed towards
inferring COVID-19 prevalence (i.e., number of positive cases in the general
population) [from wastewater sources.](https://www.sciencedirect.com/science/article/pii/S2468584420300404)

Using water-services to determine diseases has two primary benefits over
targeted testing mechanisms:

1. Data is relatively unbiased, by sick people being more likely to seek out tests
   - There is also the possibility for reducing other biases such as socio-economic as everyone makes use of these services 
2. Population wide statistics can be determined whilst being cheaper and less
   invasive than [alternative testing methods](https://www.sciencedirect.com/science/article/pii/S0043135420307181)

Current (hopefully becoming past) world events have reinvigorated interest in
using water to track diseases, specifically in using wastewater to estimate
COVID-19 cases and outbreaks. For example, before 2020 interest in wastewater
surveillance, as measured by Google trends (not the best metric, but one
nonetheless) shows spikes in internet searches for the topic.

![figures/g_trends.png](./figures/g_trends.png)


This trend is reflected in scientific publications as well, a quick search for
[wastewater disease
tracking](https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&as_ylo=2018&q=wastewater+disease+tracking++-Covid+-SARS&btnG=)
ignoring COVID as a keyword yields ~11,000 results since 2018. Whereas [not
excluding these
terms](https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&as_ylo=2018&q=wastewater+disease+tracking+&btnG=)
shows closer to ~15,000 results in the same time frame. Again, this is all just
conjecture and more robust analysis would be needed to assert anything concrete
about this trend. 

Regardless of the public and scientific interest in wastewater disease
detection the key questions are: 1) can COVID-19 particle concentration be measured in wastewater? 2) Can the measurements be used to track COVID-19 prevalence?

Well, that first question is already answered. Several countries have been
activity using wastewater as a data collection method including:
- The UK
- Australia 
- Germany
- Italy 
- Finland

Even better, some countries, such as the UK, make these [data publicly
available](https://post.parliament.uk/monitoring-wastewater-for-covid-19/). This
enables the answering of the second question, "is this data useful to monitor?".

Well, long story short, several papers have already been published and
demonstrate the usefulness of wastewater COVID monitoring for public health:

- [Controlling COVID-19 Pandemic through Wastewater Monitoring](https://www.scirp.org/journal/paperinformation.aspx?paperid=100434)
- [Wastewater monitoring outperforms case numbers as a tool to track COVID-19 incidence dynamics when test positivity rates are high](https://www.sciencedirect.com/science/article/pii/S0043135421004504)
- [Non-intrusive wastewater surveillance for monitoring of a residential building for COVID-19 cases](https://www.sciencedirect.com/science/article/pii/S0048969721024906)

However, it's 
1. Educational
2. More fun
for us to explore the publicly available data ourselves. Hence, the purpose of
this data story. 

# Getting the data
![https://zenodo.org/badge/DOI/10.5281/zenodo.7107061.svg](https://zenodo.org/badge/DOI/10.5281/zenodo.7107061.svg)

Data is stored in Zenodo under [record7107061](https://zenodo.org/record/7107061). 


In [1]:
#!wget https://zenodo.org/record/7107061/files/data.zip
#!unzip data.zip

# Setting up Python environment and data

First, we setup our data science workspace by organising which libraries we are
using. Here we are using fairly standard Python3 libraries, all of which are
available on [PyPi](https://pypi.org/)

See running and installation [file](Cleaning_and_data_info.md) for complete setup guide

In [2]:
import numpy as np 
import pandas as pd 
import plotly.express as px
from scipy.optimize import curve_fit, minimize
import json 
from glob import glob
from area import area
import matplotlib.dates as mdates
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = "notebook" 


ModuleNotFoundError: No module named 'area'

## Loading Data 

We use pandas for all of our interactions with data tables, it exponentially
makes our lives easier for working with tabular data. For example, the data is
provided by the English government in an imperfect format for data processing,
as such there are additional rows and entries of random notes and other data surrounding the
elements we are interested in. Additionally, there are multiple "sheets" of
data. Thus, when we load these data we skip a few rows and read only a specific
sheet in the document. 

The data here is given as gene copies per litre (gc/l) for number of COVID-19
RNA particles at a range of locations on various dates. 

In [ ]:
water_data = pd.read_excel("data/wastewater.ods", sheet_name='Weekly_concentrations', skiprows=4)
water_data.head()

From these data we can see that there are 3 given identifiers for sampling
location
1. Region name 
2. Site code
3. Site name

Site code and name both have the same resolution, but site name is somewhat more
friendly to read. [So really there are only 2
identifiers.](https://en.wikipedia.org/wiki/Database_normalization)

We can do some quick investigation on these data. Firstly we can count how many
wastewater testing sites there are per region in England. This allows us to
determine if there is a good distribution of these sites across geographic regions.


In [ ]:
region_count = water_data.groupby("Region name").size().reset_index().rename(columns={0:'Count'})
fig = px.bar(region_count, x='Region name', y='Count')
fig.show()

Here, we notice there are two outliers: 

1. London
   - This one can be forgiven, it is a much smaller a region than the others 
2. The North East. 
   - This region has fewer sites than the other regions. This could also be
    explained through population / area differences. 
   - This may be worth remembering and considering later in our analysis if we
    notice issues with data.

Next we can do a summary to count the total number of sites and regions. This
will help us in determining how to direct our analysis and the scope at which we
first want to focus

In [ ]:
nsites = len(water_data['Site name'].unique())
print(f"There are: {nsites} individual testing sites \n")

print("Regions in England wastewater data:")
for idx, i in enumerate(water_data['Region name'].unique()):
    print(f"\t{idx+1}: {i}")

As we can see, looking at site name or code will give us 274 sources for data
whereas for each region there are only 9 distinct groups to consider. Where each
region has between 8 and 47 sites.

For initial analysis we will look at this at the highest level — per region.
Choosing to analyse at the region level will simplify our investigation. This
will give a general insight into the relationship between wastewater COVID RNA
levels and case numbers. 

It is entirely possible, probable even, that each region will have variation
within them. For example cities compared to more rural regions could have
significant differences. However, for the sake of a broad overview we will use
regions as a starting point. Additionally, if we were to observe few differences between regions in terms of
wastewater COVID-19 RNA samples then we could contemplate considering England as
a single region and simplify analysis even further.

### Averaging data for each region 

For each of these regions we need to average the COVID-19 RNA samples per litre of water,
but also take the sum of the population all with respect to the date. Currently
the data is shaped such that each sampling date is given as a column

In [ ]:
water_data.columns

Using the `agg` function and a
`groupby` we can find the mean gene count per litre (gc/l) for each date. The
`agg` method also allows us to specify that for population we require a sum for
the regions' sites, not
an average. It is important to note that population is a dynamic variable,
however in these data the value does not change with respect to time. 

To save some long typing out of column names we can construct an aggregate
dictionary which applies a mean function to all the columns with a date and sum
function to the population. 

In [ ]:
data_aggs = {k:'mean' for k in water_data.columns[4:]}
data_aggs['Population'] = 'sum'
water_data_regions = water_data.groupby('Region name').agg(data_aggs).reset_index()
water_data_regions

As this is real data it is fully expected that there will be some invalid or
missing data. We can check for `NaN` values in our dataframe using:

In [ ]:
water_data_regions[water_data_regions.isna().any(axis=1)]

Two rows containing `NaN` values is not a huge issue. It's good to be aware of
as it could cause errors later on in our analysis.

## Loading spatial data 

Personally, I love tables of data, but sometimes visually viewing things can
really help to rationalise and compare results. For interactive plots which can
be adjusted without editing the underlying code I like to use Plotly.

Plotly allows for very pretty and very fast plotting of geography data through
the use of the [GEOJSON file format](https://geojson.org/). We use the json library and python file reader to load the
data file which contains a series of polygons which makeup UK region borders.

In [ ]:
with open("./data/uk_regions.geojson") as f:
    UK_geojson = json.load(f)

Inspecting this dataobject we can see it contains multidimensional data. In
`features` we see that there are entries with parameters for each region.
Specifically, we see that one particular entry contains data similar to our
Region name which we have in the wastewater data. 

In [ ]:
UK_geojson['features'][0]['properties']

## Matching spatial data with waste water data 

The feature named `eer17nm` matches with "Region
name" in the wastewater table. With one minor exception, the name for East of England
is slightly different, so we can rename it in the wastewater dataframe to make
plotting and analysis easier moving forward. 

In [ ]:
water_data_regions.loc[water_data_regions['Region name'] == 'East of England', 'Region name'] = 'Eastern'

To test if this match-up is successful, and if our data is sensible we make a
simple Choropleth plot using population as our information values

In [ ]:
cmap_range = (water_data_regions['Population'].min(), water_data_regions['Population'].max())
fig = px.choropleth(water_data_regions, geojson=UK_geojson, locations='Region name', color='Population', featureidkey="properties.eer17nm", range_color=cmap_range, )
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

# Analysing region features

A major red-flag to this analysis is the population of London. The previous plot
shows that for most regions the population between them reasonably similar,
however London's population is approximately 2x greater than any other
individual region. Therefore, we should think carefully about how we include
this in our analysis. It might be that the best course of action would be to
analyse London independently or perhaps include some additional parameter in any
models which include a population statistic for each region.

Permit me to interject with a bit of personal data science philosophy, I like to
think that the most exciting things happen when data stands out or doesn't
appear to make sense. Explaining these standouts gives additional purpose to our
analysis. As we go through this process we should keep a collection
of reasons for possible standouts and use them to form hypotheses about what is
going on with the data. 

## Population density 

From these data it is clear that there is one outlining area with a high
population but small area. However, this has been quantified only visually for
the area. Therefore we should try and put some numbers to this so that we have a
better idea how significant this density difference really is. Using these and
another library, `area`, we can approximate region area of objects in a GEOJSON
file. 

In order to get the area of a region we need to grab the specific polygon for
which it belongs. Here we can write a quick function to perform this lookup.
Computationally it is very inefficient as we have to loop through all polygons
in the GEOJSON file and also perform a comparison between region names to
evaluate if it is the correct one. However, computational effort is cheap and
our time is not, so we use it as is.

With this we perform an apply function in pandas to create a new column in our
data table. From this we calculate a Population Per Area (PPA). In the apply
function we make use of a `lambda` function, this can be thought of as a small,
single expression function which returns the result of said expression.


In [ ]:
# This is incredibly inefficient, but computational power is cheap and developer
# time is not! 

def lookup_district_area(district):
    "Gets district area in squared meters"
    for poly in UK_geojson['features']:
        if poly['properties']['eer17nm'] == district:
            return area(poly['geometry'])
    return -1 # Something broke

water_data_regions['Area'] = water_data_regions.apply(lambda x: lookup_district_area(x['Region name']), axis=1)

water_data_regions['PPA'] = water_data_regions['Population']/water_data_regions['Area']

water_data_regions[['Region name','Area', 'Population', 'PPA']]

As expected, and unsurprisingly the PPA of the London region is orders of
magnitude greater than any other regions of England

In [ ]:
fig = px.bar(water_data_regions, x='Region name', y='PPA')
fig.show()

From these population per area (PPA) results a very strong case could be made to
consider the London region as a separate and distinct area when performing
further analysis. Particularly when disease dynamics are different in highly
populated areas this could cause issues when making comparisons with London and
the other regions of England <sup>[citation needed]</sup>

# COVID-19 gene counts per litre over time in England's regions

For epidemics we are often more interested in how the disease is changing over
time. An individual time point cannot tell us if the disease is becoming more or
less prevalent. Therefore we next want to organise our data in such a way that
we can compare gc/l and how they change in each region overtime. 

Currently, the data is in a format where columns exist for each date entry. Our
target is to have a format where `Date` is a single column. 

In [ ]:
water_data_regions.columns[4:-3]

With the data available as of August 2022 we have 29 entries which run from
22/06/2021 - 04/01/2022. 

Again, I'd like to
take this opportunity to express my love for pandas as this form of data
wrangling is completed in a single beautiful line of code. 

In [ ]:
water_data_dates = water_data_regions.melt(id_vars=['Region name','Population','Area','PPA'], var_name='Date', value_name='gc/l')
water_data_dates['Date'] = pd.to_datetime(water_data_dates['Date'], dayfirst=True)
water_data_dates

At this point we can ask the question: how large is the variation of gc/l
measurements between England's regions? 

To answer this, and to decide if we can take a country-scale look at the data or
should stick with region scale, we can perform another aggregation of the data
to get the mean behaviour of England's wastewater values.

In [ ]:
water_data_dates_mean = water_data_dates.groupby('Date').agg( {'gc/l':'mean'} ).reset_index().sort_values(by='Date')
water_data_dates_mean.head()

### Is the mean a good approximation for gc/l or does each region vary greatly?

We then plot these averaged data against the region specific data. 

In [ ]:
fig = px.line(water_data_dates, x='Date', y='gc/l', color='Region name')
fig.update_traces(opacity=0.4, line_width=1)
water_data_dates_mean['Region name'] = 'mean' # adding this column for label in legend
fig.add_trace(px.line(water_data_dates_mean, x='Date', y='gc/l', color='Region name').data[0])
fig.show()

Visually, the mean values for gc/l appear to follow a general trend over time
for most of the regions, there are some notable disagreements around August 2021
and January 2022. Therefore, we next will investigate and quantify how large
these differences are between regions and England's mean gc/l for each date. 
What about the percentage difference from the mean?


To match across the two dataframes we make a lookup dictionary for the mean
values. We also define a lambda function `pc_diff_calc` which calculates the
percentage difference between values `a` and `b`. This gives a simple metric by
which we can see if any particular dates and or regions greatly differ from mean
gc/l estimates. 

In [ ]:
mean_per_dates = {water_data_dates_mean['Date'][i]:water_data_dates_mean['gc/l'][i] for i in range(len(water_data_dates_mean))}
pc_diff_calc = lambda a,b: abs((a - b) / b) * 100.0
water_data_dates['Difference to mean'] = water_data_dates.apply(lambda x: pc_diff_calc(x['gc/l'], mean_per_dates[x['Date']] ), axis=1 )

If there was a strong agreement between the mean estimate and the results in
each region then we would expect the average of the values to be close to 0

In [ ]:
mean_differences = water_data_dates.groupby('Region name').mean()['Difference to mean'].reset_index().sort_values(by='Difference to mean')

In [ ]:
fig = px.bar(mean_differences, x='Region name', y='Difference to mean')
fig.show()

Curiously there seems to be a gradient of results here. With no single region
standing out. Even our predicted outlier of London is comparable with another
region in terms of it's  disagreement to the average gc/l.

To explore more closely where these differences arise, and when, we plot the
same data over time.

In [ ]:
fig = px.line(water_data_dates, x='Date', y='Difference to mean', color='Region name')
fig.show()

From plotting these data we can see that whilst initially the general trend
seemed similar between each date and region with the mean estimate. However,
there are a number of extreme values which would suggest that averaging at a
country level is not going to yield true insights into gc/l trends in England
overtime. 

One additional check we can do is quantify and view the sum of these
disagreements. This would show us two things. Firstly, if any regions are close
to matching the mean or if they are, either consistently or from outliers,
largely different from the mean estimate. Secondly, it can allow us to make a
rough comparison between the regions themselves.

## Creating an interactive spatial-temporal map for gc/l data

As previous discussed there are some missing data, data also have some
inconsistencies in that they are not all reported on the same day. We resolve
this by grouping data to the nearest month and dropping the day component (this
also will simplify downstream analysis when we join with additional data).

In [ ]:
water_data_pm = water_data_dates.groupby([pd.Grouper(key='Date', freq='M'), 'Region name'])['gc/l'].mean().reset_index()
water_data_pm['Date'] = water_data_pm['Date'].dt.strftime('%m/%Y')
water_data_pm

With plotly express we can create a chropleth plot of England and add a date
slider to view the change in gc/l over time. These plots are showing the same
data we just analysed, but in a more visually attractive style!

Re-plotting the same data in a different style can be useful, for example here
it makes the increase of gc/l seen in July and August 2021 stand out!

In [ ]:
cmap_range = (water_data_pm['gc/l'].min(), water_data_pm['gc/l'].max())
fig = px.choropleth(water_data_pm, geojson=UK_geojson, locations='Region name', color='gc/l', featureidkey="properties.eer17nm", animation_frame="Date", range_color=cmap_range)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

# Incorporating COVID-19 reported cases data

Now, just a reminder, what we are truly interested in is whether or not we can
use these data to estimate COVID-19 cases. Therefore, we still have a major
component of data missing - case numbers. Once again, the UK government makes
these data open and freely available. 

We wrangle these data with some pre-processing steps to make it easier to work
with. Like our previous analysis there is a discrepancy in the naming of East of
England, thus we can fix that here. 

Whilst we are tweaking the data we perform another rounding of dates. This way
we have case numbers aligned to weeks, we convert this to nearest month and therefore can be directly tied into
our wastewater data.


In [ ]:
covid_cases_by_region = pd.read_csv("./data/covid_cases/covid_data_combined.csv")
covid_cases_by_region.loc[covid_cases_by_region['areaName'] == 'East of England', 'areaName'] = 'Eastern'
covid_cases_by_region['date'] = pd.to_datetime(covid_cases_by_region['date']).dt.strftime('%m/%Y')
covid_cases_by_region = covid_cases_by_region.groupby(['areaName','date']).sum().reset_index().drop(['Unnamed: 0'], axis=1)
covid_cases_by_region.sample(5)

To link these data we use the merge function of pandas. We create a new
dataframe merging on the date and region name values of both tales of data.

In [ ]:
cases_water_pm = pd.merge(water_data_pm, covid_cases_by_region, right_on=['date', 'areaName'], left_on=['Date', 'Region name'])
cases_water_pm.sample(5)

Without looking at the data it would be a sensible hypothesis to state that gc/l
of COVID-19 RNAs in wastewater is somewhat proportional to the number of positive
cases detected. We can quick and visually explore this idea with a dual axes
plot of the two values.

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=cases_water_pm['Date'], y=cases_water_pm['gc/l'], name="gc/l"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=cases_water_pm['Date'], y=cases_water_pm['newCasesBySpecimenDate'], name="Cases by date"),
    secondary_y=True)

# Add figure title
fig.update_layout(
    title_text="gc/l vs. cases"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="gc/l", secondary_y=False)
fig.update_yaxes(title_text="Cases by specimen date", secondary_y=True)

fig.show()

This first look isn't convincing. General trends of up/down at the same time
seem to be correct, but the magnitude is poorly reflected. For example the cases
in Summer 2021 are not so different with Autumn, but there's a significant
upsurge of gc/l. Data agree much better in winter also. 

Important to note here is that we are considering all regions lumped together.
Hence the error bars shown in this plot. Our next step would be to consider this
in the per-region scope. It would be interesting to consider if some regions'
case numbers are
more accurately represented by wastewater gc/l than others. 

In [ ]:
specs=[ [ {"secondary_y": True} for _ in range(3)] for _ in range(3)  ]
fig = make_subplots(rows=3, cols=3,  subplot_titles=cases_water_pm['Region name'].unique())

#normalise a bit for plotting
cases_water_pm['newCasesBySpecimenDateNorm'] = cases_water_pm['newCasesBySpecimenDate'] / cases_water_pm['newCasesBySpecimenDate'].max()
cases_water_pm['gc/lNorm'] = cases_water_pm['gc/l'] / cases_water_pm['gc/l'].max()

leg = True
make_trace_cases = lambda rn, leg: go.Scatter(x=cases_water_pm[cases_water_pm['Region name'] == rn ]['Date'], y=cases_water_pm[cases_water_pm['Region name'] == rn ]['newCasesBySpecimenDateNorm'], name="Cases by date", marker = {'color' : 'red'}, showlegend=leg,)
make_trace_gcl = lambda rn, leg: go.Scatter(x=cases_water_pm[cases_water_pm['Region name'] == rn ]['Date'], y=cases_water_pm[cases_water_pm['Region name'] == rn ]['gc/lNorm'], name="gc/l", marker = {'color' : 'blue'}, showlegend=leg)

for idx in range(3):
    for idy in range(3):
        i = idx*3 + idy
        rn = cases_water_pm['Region name'].unique()[i]
        t1 = make_trace_cases(rn, leg)
        t2 = make_trace_gcl(rn, leg)
        fig.append_trace(t1, idx+1, idy+1)
        fig.append_trace(t2, idx+1, idy+1)


fig['layout'].update(autosize= False, 
              width= 2*800, 
              height= 800, 
              showlegend=leg,
              hovermode='x')

fig.update_xaxes(tickangle=45)

names = set()
fig.for_each_trace(
    lambda trace:
        trace.update(showlegend=False)
        if (trace.name in names) else names.add(trace.name))

fig.show()

Interestingly, most of these regions have a vaguely similar trend of both gc/l
and case numbers. 

# Can we use the gc/l data to predict covid cases? 

Let's chose a specific area to use as a 'case study' and to focus our analysis. Having spent the past 4 years in East Anglia I am biased to choose the 'Eastern'
region for an initial test. So, if we kept up with case reporting during the COVID-19 pandemic then we know that
case numbers typically follow an exponential curve which is normally given as 

$$ y = m^{bx} $$

where $y$ is the output variable, $m,b$ are coefficients and $x$ as a
dependant variable. The coefficient are set by using a curve fit function which
finds the optimal values for $m,b$ which when used with $x$ give the most
accurate estimation of $y$ (which is possible with the given function). 
These values for $m,b$ are stored in `popt`. 

We can show that this kind of model is a relatively good
approximation for COVID-19 cases over time by fitting this function to the data
we have available. 

A few key things to note: 
1. We convert our dates to a `datenum` with the mdates class of matplotlib - this gives them a floating point representation.
2. $x$ will be this `datenum` value and our $y$ is COVID-19 cases.
3. $\hat{y}$ is our approximation of case numbers for a given date.
4. These models have the clear limitation of not being limited. That is to say that they have no cap and eventually become wrong. 
   - e.g. Dates next year are likely predicting more cases than humans currently alive. 
5. We use `start` as a variable to make curve fitting more accurate.
   - The curve fitting function is blind
   to what the optimal values should be and has a search space of -inf to +inf,
   thus we can speed up the process by specifying an approximate value to begin
   with in the `start` variable. 


To evaluate if we can predict prior case numbers as well as future we have
looked a month ahead and behind but only fit the data from June to December 2021.
Please do note and understand that if we used a different date range this
analysis would very different. I intentionally use this range to avoid the
effects of the UK's implementation of the so-called [Plan
B](https://www.gov.uk/government/news/prime-minister-confirms-move-to-plan-b-in-england)
to halt infection spread. 

In [ ]:

def exp_f(x, m, b):
    return m*np.exp(b*x)

# One solution to working with dates is to consider them numerically 
# This may not be an ideal way of working with dates but it works for these simple
# analysis 

cases_water_pm['DateNum'] = mdates.date2num(pd.to_datetime(cases_water_pm['Date']))
eastern_cases_water = cases_water_pm[cases_water_pm['Region name'] == 'Eastern' ]
x1 = eastern_cases_water['DateNum'].values[1:-1]
y1 = eastern_cases_water['newCasesBySpecimenDate'][1:-1]
start = 1e-3

# optimal values for parameters and their estimated covarience
# We're really only interested in popt for now.
popt, pcov = curve_fit(exp_f, x1, y1, p0=[start,start], maxfev=50000)

covid_cases_by_region['date'] = pd.to_datetime(covid_cases_by_region['date'])
covid_cases_by_region = covid_cases_by_region[(covid_cases_by_region['date'] > '2021-05') & (covid_cases_by_region['date'] <= '2022-01') ]
covid_cases_by_region = covid_cases_by_region[covid_cases_by_region['areaName'] == 'Eastern'].sort_values(by='date')

X_nu = mdates.date2num(covid_cases_by_region['date'])
y_nu = covid_cases_by_region['newCasesBySpecimenDate']
X_approx = np.linspace(X_nu.min(), X_nu.max(), num=1000)

y_hat = exp_f(X_approx, *popt)


fig = go.Figure(data=go.Scatter(x=mdates.num2date(X_nu), y=y_nu, name='Number of new positive cases'))
fig.add_trace(go.Scatter(x=mdates.num2date(X_approx), y=y_hat, name='Exponential Curve Fit'))
fig.add_trace(go.Scatter(x=mdates.num2date(x1), y=y1, name='Training values'))

fig.show()

This looks like a very good fit, although it's more useful if we can get a
numerical quantification of this fit. A simple method for this is to calculate
an $R^2$ value. The $R^2$ is essentially a "goodness-of-fit" measure, it looks
at the variance explained by the model with respect to the total variance. Thus,
values for $R^2$ close to 1 are considered better than those close to 0. 

In [ ]:
def calc_r2(X, popt, y1, f):
    y_fit = f(X, *popt)
    ss_res = np.sum((y1 - y_fit) ** 2)
    ss_tot = np.sum((y1 - np.mean(y1)) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    return r2

r2_exp_cases = calc_r2(X_nu, popt, y_nu, exp_f)
print(f'R^2: {r2_exp_cases}')

This is somewhat a loaded question given that we have already made some
observations about the data, but we next could ask "does wastewater gc/l follow
a similar curve". The logic being that gc/l in wastewater must be proportional
to case numbers and therefore would have a similar exponential trend. 

To do this we first estimate the relationship between gc/l and case numbers, we
then can re-use the initial model to using predictions from the gc/l -> case
numbers. 

In [ ]:
y2 = eastern_cases_water['gc/l'][1:-1]
popt, pcov = curve_fit(exp_f, x1, y2, p0=[start,start], maxfev=50000)
y3 = eastern_cases_water['gc/l']
x3 = eastern_cases_water['DateNum']
y_hat = exp_f(X_approx, *popt)

fig = go.Figure(data=go.Scatter(x=mdates.num2date(x3), y=y3, name='gc/l'))
fig.add_trace(go.Scatter(x=mdates.num2date(X_approx), y=y_hat, name='Exponential Curve Fit'))
fig.add_trace(go.Scatter(x=mdates.num2date(x1), y=y2, name='Training values'))

fig.show()

In [ ]:
r2_exp_gcl = calc_r2(x3, popt, y3, exp_f)
print(f'R^2: {r2_exp_gcl}')

Completely unsurprisingly we can see that these data are not so easily modelled
as case numbers are. The negative value of R^2 indicates that this is a model so
bad that a horizontal line $y=x+m$ would be a better estimator! 
Therefore it is clear that if a relationship exists between gc/l and case numbers
then it is likely a more complex relationship.

Nonetheless, we can still attempt to use wastewater gc/l values to build an
estimator in a similar fashion as above.

In [ ]:
popt, pcov = curve_fit(exp_f, y2, y1, p0=[start,start], maxfev=50000)
y1_est = exp_f(y2, *popt)

popt, pcov = curve_fit(exp_f, x1, y1_est, p0=[start,start], maxfev=50000)
y_hat = exp_f(X_approx, *popt)

fig = go.Figure(data=go.Scatter(x=mdates.num2date(X_nu), y=y_nu, name='Number of new positive cases'))
fig.add_trace(go.Scatter(x=mdates.num2date(X_approx), y=y_hat, name='Exponential Curve Fit'))
fig.add_trace(go.Scatter(x=mdates.num2date(x1), y=y1, name='Training values'))

fig.show()

In [ ]:
r2_exp_gcl_to_cn = calc_r2(X_nu, popt, y_nu, exp_f)
print(f'R^2: {r2_exp_gcl_to_cn}')

We can next ask "can date and gc/l be used to increase
accuracy of this model?". 

We use a similar function as before, but modify it slightly to accept more
determining variables in the form of $x_i$

$$y = m^{bx_1} + x_2$$

In [ ]:
def exp_f2(X, m, b, c):
    return m*np.exp(b*X[0]) * X[1]*c

X = np.vstack((x1,y2))
popt, pcov = curve_fit(exp_f2, X, y1, p0=[start,start,start], maxfev=50000)

X_approx = np.dstack((np.linspace(X_nu.min(), X_nu.max(), num=1000), np.linspace(X[1].min(), X[1].max(), num=1000)))[0].T
y_hat = exp_f2(X_approx, *popt)

fig = go.Figure(data=go.Scatter(x=mdates.num2date(X_nu), y=y_nu, name='Number of new positive cases'))
fig.add_trace(go.Scatter(x=mdates.num2date(X_approx[0]), y=y_hat, name='Exponential Curve Fit'))
fig.add_trace(go.Scatter(x=mdates.num2date(x1), y=y1, name='Training values'))

fig.show()

In [ ]:
X_2 = np.vstack((X_nu, x3))
r2_exp_cases_gcl = calc_r2(X_2, popt, y_nu, exp_f2)
print(f'R^2: {r2_exp_cases_gcl}')

In [ ]:
print(f"R^2 value for model predicting cases with exponential function: {r2_exp_cases}")
print(f"R^2 value for model predicting gc/l with exponential function: {r2_exp_gcl}")
print(f"R^2 value for model predicting cases using case # and gc/l with exponential function: {r2_exp_cases_gcl}")

Interestingly incorporating the wastewater data does not harm the predictive
power of this exponential model. It doesn't make it *much* better, but it doesn't make
it worse. Well, one could argue that simple models i.e., fewer parameters, are
better overall. 

From here we can think about reasons why this wastewater data is not correlating
well with COVID-19 test cases. Blaming testing bias, vaccination rates or some
other factor could be a viable answer. Another could be the effect of weather,
specifically rainfall. Rainfall in England is highly variable over the course of
the year and as such could result in the concentration of COVID-19 RNA particles
to be higher or lower dependent on rainfall. 

# Weather data 

Like all the data used thus far weather data in the UK is also relatively
easy to come by and is in a semi-accessible format. The weather data is extensive and spans more than a century of monthly
data. The data we are loading in here has been pre-processed according to the
`Cleaning_and_data_info.md` file. 

Firstly, we load in the data and can remove data prior to June 2021.

In [ ]:
rainfall_files = glob("./data/rainfall/*.csv")
rainfall_files

As each region has been given its own datafile we save the filenames into a
dictionary. We can then use this dictionary to hold each individual file until we
match it up with the data we've been using previously for wastewater/COVID-19 cases.

In [ ]:
rainfall_dfs = {}
for rff in rainfall_files:
    rff_name = rff.split('/')[-1].split('.')[0]
    rainfall_dfs[rff_name] = pd.read_csv(rff)[-4:]
rainfall_dfs.keys()

## Matching rainfall to wastewater locations

Unfortunately the names and locations used for the rainfall measurements are not
ideal, thus the quickest and easiest solution is to manually create a dictionary
to look up and match the data 

Below we have two maps, the left showing how the weather data are labelled by the
UK MET office, the right on how our case data and wastewater data are roughly
categorised. 

| Met office regions| Wastewater regions|
|---|---|
| ![Met](https://www.metoffice.gov.uk/binaries/content/gallery/metofficegovuk/images/weather/learn-about/past-uk-weather-events/uk-regional-boundaries/districtregions.gif/districtregions.gif/metofficegovuk%3Axsmall) | ![link](http://projectbritain.com/regions/images/regions.png) |





It is crucial to note that these are very rough approximations based on visually
assigning a close matching region. It is likely that a more robust
process is possible and that more local measurements for rainfall is available
(but a project in and of itself).

In [ ]:
# These are rough approximations to the regions
region_to_rainfall_location = {
'North West':'England_NW_and_N_Wales',
'South East' : 'England_SE_and_Central_S',
'South West' : 'England_SW_and_S_Wales',
 'London':'England_SE_and_Central_S',
 'West Midlands' : 'Midlands', 
 'Yorkshire and The Humber': 'England_E_and_NE',
   'North East':'England_E_and_NE',
    'East Midlands':'Midlands',
      'Eastern':'East_Anglia'
}

In [ ]:
rainfall_dfs[rff_name][rainfall_dfs[rff_name].columns[:-5]].head()

The shape and format of these rainfall data need altered to work with our
current setup. Similar to before when we had date column headings we now have
each row representing an individual year. This time we perform a melt function
to convert these columns into a single column. We combine this column with the
existing year column to build a year/month string which can be formatted by
pandas to %m/%y format like the rest of our data.

We repeat this process for each region, create a region ID column and
concatenate all of the dataframes. 

In [ ]:
rf_rain_pm_lst = []
for rff_name in rainfall_dfs.keys():
    rf_melted = rainfall_dfs[rff_name][rainfall_dfs[rff_name].columns[:-5]].melt(id_vars='year', value_name='rainfall')
    rf_melted['Date'] = pd.to_datetime(rf_melted.apply(lambda x: f"{x['year']}/{x['variable']}", axis=1)).dt.strftime('%m/%Y')
    rf_melted['Location'] = rff_name
    rf_rain_pm_lst.append(rf_melted)
rf_rain_pm = pd.concat(rf_rain_pm_lst)
rf_rain_pm = rf_rain_pm.drop(['variable','year'], axis=1)
rf_rain_pm.sample(10)

Now the data is in a workable format for us we can go about adding it to our
existing dataframe. We use the dictionary created above for matching regions
between rainfall data and the wastewater to match these up. From this we can use
a lookup function to get the rainfall on a particular date in a specific region. 

In [ ]:
cases_water_pm['rainfall_region'] = cases_water_pm.apply(lambda x: region_to_rainfall_location[x['Region name']], axis=1)

def lookup_date_region_rain(date, region):
    row = rf_rain_pm.loc[(rf_rain_pm["Date"]==date)&( rf_rain_pm["Location"]==region)]['rainfall']
    if len(row) == 0:
        return 'n/a'
    return row.values[0]

lookup_rf = lambda x: lookup_date_region_rain(x['Date'], x['rainfall_region'])
cases_water_pm['Rainfall'] = cases_water_pm.apply(lookup_rf, axis=1)
cases_water_pm

Before we attempt to use this data in our models we can first have a look at the
general trend for these variables: 
1. Wastewater gc/l
2. Rainfall (mm)
3. COVID-19 positive case numbers

Keeping it simple to start off with we will average and group by date, thereby
ignoring the spatial component of the data.

In [ ]:
rain_vs_gcl = cases_water_pm.groupby(['Date'])[['gc/l', 'Rainfall', 'newCasesBySpecimenDate']].mean().reset_index()
rain_vs_gcl

These three columns are widely different and do not follow the same scale. To
plot them together and make a visual observation of trends over time we perform
a simple normalisation. Each column is divided by its maximum value which
results in values for each $\in [0,1]$

In [ ]:
rain_vs_gcl['gc/lNorm'] = rain_vs_gcl['gc/l']/ rain_vs_gcl['gc/l'].max()
rain_vs_gcl['rainfallNorm'] = rain_vs_gcl['Rainfall']/ rain_vs_gcl['Rainfall'].max()
rain_vs_gcl['newCasesBySpecimenDateNorm'] = rain_vs_gcl['newCasesBySpecimenDate']/ rain_vs_gcl['newCasesBySpecimenDate'].max()

rain_vs_gcl_melted = rain_vs_gcl[['Date', 'gc/lNorm', 'rainfallNorm', 'newCasesBySpecimenDateNorm']].melt(id_vars='Date')

fig = px.line(rain_vs_gcl_melted, x='Date', y='value', color='variable')
fig.show()


# Correcting wastewater values using rainfall data

We can reason out that wastewater data for COVID-19 gc/l is somehow affected by
the rain. If we hypothesise that this is a linear relationship i.e., gc/l is in
someway proportional to rain amount then it may be possible that gc/l values, if
scaled correctly, could help give more accurate case number predictions!

With this in mind our model becomes: 

$$y = a^{bx_d} cx_wx_r$$

where $x_d$ is date, $x_w$ wastewater gc/l and $x_r$ the rain measurement for
that date, $a,b,c$ are fitted coefficients.




In [ ]:
east_pm = cases_water_pm[cases_water_pm['Region name'] == 'Eastern'].copy()
east_pm['DateNum'] = mdates.date2num(pd.to_datetime(east_pm['Date']))

X_nu = east_pm['DateNum'] 
y_nu = east_pm['newCasesBySpecimenDate']
X_approx = np.linspace(X_nu.min(), X_nu.max(), num=1000)
y1 = east_pm['newCasesBySpecimenDate'][1:-1]

y2 = east_pm['gc/l'][1:-1]
x2 = east_pm['DateNum'][1:-1]
x3 = east_pm['Rainfall'][1:-1]

y2_f = east_pm['gc/l']
x2_f = east_pm['DateNum']
x3_f = east_pm['Rainfall']


In [ ]:
def exp_f3(X,m, d, w):
    return m*np.exp(d*X[0]) * (X[1]*X[2]*w)

X = np.vstack((x2,y2,x3))
popt, pcov = curve_fit(exp_f3, X, y1, p0=[start, start,1], maxfev=100000)


In [ ]:
X_approx = np.dstack((np.linspace(X_nu.min(), X_nu.max(), num=1000),
                      np.linspace(X[1].min(), X[1].max(), num=1000), 
                      np.linspace(X[2].min(), X[2].max(), num=1000)))[0].T
y_hat = exp_f3(X_approx, *popt)

fig = go.Figure(data=go.Scatter(x=mdates.num2date(X_nu), y=y_nu, name='Number of new positive cases'))
fig.add_trace(go.Scatter(x=mdates.num2date(X_approx[0]), y=y_hat, name='Exponential Curve Fit'))
fig.add_trace(go.Scatter(x=mdates.num2date(x1), y=y1, name='Training values'))

fig.show()

In [ ]:
X_3 = np.vstack((x2_f,y2_f,x3_f))
calc_r2(X_3, popt, y_nu, exp_f3)
print(f'R^2: {calc_r2(X_3, popt, y_nu, exp_f3)}')

# Conclusion & Discussion




### The simple exponential model using rainfall and gc/l to estimate case numbers is unsuitable

From the previous model and figure we can see that the model using gc/l and
rainfall is not terrible, but it isn't great either. Perhaps most encouraging is
that from the models prior we determined that gc/l could not provide any useful
results when used alone, the inclusion of rainwater giving some increase in
accuracy could be encouraging (it could also be something quite obscure
happening also!). 

From our own results here we cannot discount the use of wastewater data and
weather data for use in predicting disease case numbers, but it is clear that
the models we have chosen need refined or perhaps completely rethought. 


### Looking at these models in different points in time and space

As discussed in the beginning we average firstly by time (monthly) and then
again spatially (by England regions. We then focused a lot of the models on a
specific region of England. The logical future work, which shouldn't be too
difficult would be to explore these models, with minimal change, on the other
regional data we currently have. It would be interesting to see if any areas are
better/worse performing for the model. 

Additionally, the data we have decided to use is very constrained in terms of
time, a month is a significant amount of time for something which grows
exponentially like disease case numbers. Slightly better data wrangling could
easily allow us to repeat all analysis with roughly ~4x the data points if we
consider the problem weekly rather than monthly. Once again, this better
resolution may change our outlook/result from the current models. 

### Wastewater gc/l could be used to monitor disease trends

As we observed when initially compiling and organising the data, there is a
rough agreement, albeit in shape not magnitude, of case numbers and wastewater
gc/l. This could suggest that these data may be suitable as “warning” systems
rather than predictors of case numbers. 

Therefore, these data could be used to target areas which may need
interventions, such as increased disease testing, tracking/tracing or other
appropriate responses. 

### The ethics of using wastewater data

One of the major strengths, and desire for these data to be used, is that they
are relatively unbiased. Everyone who lives in an area contributes to wastewater
and therefore doesn't have the issue of ill people being more prone to being
tested and recorded. However, some concerns have been raised over the ethics of
this level of monitoring of people without explicit consent. [Gableetal.](https://academic.oup.com/jlb/article/7/1/lsaa039/5861905),
for example, suggest that these data sources be treated similarly to direct
testing methods (e.g. PCR/LFT). One could make the argument that the privacy of
individuals is at risk when performing these analysis. 

